<a href="https://colab.research.google.com/github/SheikhMudassarHanif/NLP/blob/main/TextSummariztionUsingLLMipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Using BART MODEL for Text Summarization.


**.   Simple BERT ISNT TRAINED TO GENERATE TEXT**




loading the model from hugging face

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,AdamW,Seq2SeqTrainer,Seq2SeqTrainingArguments

usingmodel="facebook/bart-base"
model = AutoModelForSeq2SeqLM.from_pretrained(usingmodel)
tokenizer = AutoTokenizer.from_pretrained(usingmodel)

# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# from transformers import BertForSequenceClassification
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased')



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [4]:
model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [5]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/samsum-test.csv")
df.head()

,id,dialogue,summary
0,13862856,"Hannah: Hey, do you have Betty's number?\nAman...",Hannah needs Betty's number but Amanda doesn't...
1,13729565,Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric:...,Eric and Rob are going to watch a stand-up on ...
2,13680171,"Lenny: Babe, can you help me with something?\r...",Lenny can't decide which trousers to buy. Bob ...
3,13729438,"Will: hey babe, what do you want for dinner to...",Emma will be home soon and she will let Will k...
4,13828600,"Ollie: Hi , are you in Warsaw\r\nJane: yes, ju...",Jane is in Warsaw. Ollie and Jane has a party....


In [32]:
print(df['dialogue'][3])

Will: hey babe, what do you want for dinner tonight?
Emma:  gah, don't even worry about it tonight
Will: what do you mean? everything ok?
Emma: not really, but it's ok, don't worry about cooking though, I'm not hungry
Will: Well what time will you be home?
Emma: soon, hopefully
Will: you sure? Maybe you want me to pick you up?
Emma: no no it's alright. I'll be home soon, i'll tell you when I get home. 
Will: Alright, love you. 
Emma: love you too. 


Structing the provided dataset into hugging face dataset format using the library of datasets

In [7]:
from datasets import Dataset
train_df=df.sample(frac=0.8,random_state=42)
val_df=df.drop(train_df.index)
# train_df.head()

train_dataset=Dataset.from_pandas(train_df[['dialogue','summary']])
val_dataset=Dataset.from_pandas(val_df[['dialogue','summary']])
train_dataset=train_dataset.rename_columns({'dialogue':'source_text','summary':'summary_text'})
val_dataset=val_dataset.rename_columns({'dialogue':'source_text','summary':'summary_text'})

Tokening the inputs and labels

In [8]:
def Tokenization(examples):
    inputs = tokenizer(examples["source_text"], max_length=512, truncation=True, padding="max_length")
    targets = tokenizer(examples["summary_text"], max_length=128, truncation=True, padding="max_length")
    # Ensure all token IDs in labels are within the vocab size
    targets["input_ids"] = [
        [token_id if token_id < tokenizer.vocab_size else tokenizer.pad_token_id for token_id in target]
        for target in targets["input_ids"]
    ]
    inputs["labels"] = targets["input_ids"]
    return inputs


In [9]:
tokenized_train_dataset=train_dataset.map(Tokenization,batched=True)
tokenized_val_dataset=val_dataset.map(Tokenization,batched=True)

Map:   0%|          | 0/655 [00:00<?, ? examples/s]

Map:   0%|          | 0/164 [00:00<?, ? examples/s]

In [17]:
# val_dataset[0],tokenized_val_dataset[0]
# !pip install evaluate
# !pip install rouge_score

In [23]:
from evaluate import load

rouge_metric = load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace out-of-range token IDs in labels
    labels = [
        [token_id if token_id < tokenizer.vocab_size else tokenizer.pad_token_id for token_id in label]
        for label in labels
    ]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute Rouge scores
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Return directly if the results are raw scores
    if isinstance(result[list(result.keys())[0]], float):  # Check if the values are floats
        return result
    else:
        # For structured results, extract 'fmeasure' or other metrics
        return {key: value['fmeasure'] for key, value in result.items()}


Now inputs is tokenized now for model

In [24]:
training_args=Seq2SeqTrainingArguments(
    output_dir="./Bert_results",
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,
    # num_train_epochs=1
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    logging_steps=10,
    eval_steps=10

)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
     data_collator=data_collator,
)

<ipython-input-25-b269c49e7386>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.314300,0.431794,0.416781,0.193974,0.350951,0.351615
2,0.307800,0.413815,0.425072,0.204197,0.367706,0.367575
3,0.243600,0.417981,0.422300,0.198686,0.361141,0.361961


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now depre

TrainOutput(global_step=1965, training_loss=0.28081190567890196, metrics={'train_runtime': 413.5222, 'train_samples_per_second': 4.752, 'train_steps_per_second': 4.752, 'total_flos': 599066070220800.0, 'train_loss': 0.28081190567890196, 'epoch': 3.0})

In [27]:
bert_result=trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

Result of bert

In [28]:
bert_result

{'eval_loss': 0.4179810881614685,
 'eval_rouge1': 0.42230008994819423,
 'eval_rouge2': 0.19868585642128814,
 'eval_rougeL': 0.36114109047526505,
 'eval_rougeLsum': 0.3619607323580537,
 'eval_runtime': 51.6717,
 'eval_samples_per_second': 3.174,
 'eval_steps_per_second': 3.174,
 'epoch': 3.0}

In [33]:
def summarize_text(text, model, tokenizer, max_length=512, num_beams=4):
    # Tokenize the input
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", truncation=True, max_length=max_length)

    # Move inputs to the same device as the model
    device = next(model.parameters()).device  # Get the model's device
    inputs = inputs.to(device)  # Move inputs to the device

    # Generate the summary
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_beams=num_beams,
        early_stopping=True
    )

    # Decode the generated summary
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

# Example usage
input_text = """Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye """
summary = summarize_text(input_text, model, tokenizer)
print("Summary:", summary)
print('/n')
input_text = """Will: hey babe, what do you want for dinner tonight?
Emma:  gah, don't even worry about it tonight
Will: what do you mean? everything ok?
Emma: not really, but it's ok, don't worry about cooking though, I'm not hungry
Will: Well what time will you be home?
Emma: soon, hopefully
Will: you sure? Maybe you want me to pick you up?
Emma: no no it's alright. I'll be home soon, i'll tell you when I get home.
Will: Alright, love you.
Emma: love you too. """
summary = summarize_text(input_text, model, tokenizer)
print("Summary:", summary)




Summary: Hannah doesn't have Betty's number. Amanda will contact Larry.
/n
Summary: Emma will be home soon. Will will pick her up.


#GPT2

In [36]:
import warnings
warnings.filterwarnings('ignore')

In [38]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

In [39]:
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [40]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,  # Low-rank dimension
    lora_alpha=32,  # LoRA alpha
    lora_dropout=0.1  # Dropout
)
model = get_peft_model(model, lora_config)


In [102]:
import pandas as pd
import numpy as np
data=pd.read_csv("/content/samsum-test.csv")
data.head()

,id,dialogue,summary
0,13862856,"Hannah: Hey, do you have Betty's number?\nAman...",Hannah needs Betty's number but Amanda doesn't...
1,13729565,Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric:...,Eric and Rob are going to watch a stand-up on ...
2,13680171,"Lenny: Babe, can you help me with something?\r...",Lenny can't decide which trousers to buy. Bob ...
3,13729438,"Will: hey babe, what do you want for dinner to...",Emma will be home soon and she will let Will k...
4,13828600,"Ollie: Hi , are you in Warsaw\r\nJane: yes, ju...",Jane is in Warsaw. Ollie and Jane has a party....


In [42]:
# print(df['dialogue'][3])

Structing the provided dataset into hugging face dataset format using the library of datasets

In [108]:
import torch
from datasets import Dataset

# Split the data into training and validation sets
train_df = data.sample(frac=0.8, random_state=42)
val_df = data.drop(train_df.index)

# Create Dataset objects
train_dataset = Dataset.from_pandas(train_df[['dialogue', 'summary']])
val_dataset = Dataset.from_pandas(val_df[['dialogue', 'summary']])

# Rename columns to match the expected input/output format for the model
train_dataset = train_dataset.rename_columns({'dialogue': 'source_text', 'summary': 'summary_text'})
val_dataset = val_dataset.rename_columns({'dialogue': 'source_text', 'summary': 'summary_text'})

# Preprocessing function
def preprocess_function(examples):
    # Add "summarize: " to input dialogues
    inputs = [f"summarize: {dialogue}" for dialogue in examples["source_text"]]
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding=True)  # Adjusted max_length

    # Tokenize summaries (labels) with appropriate truncation and padding
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary_text"], max_length=128, truncation=True, padding=True  # Consistent padding
        )

    # Replace pad_token_id in labels with -100 to ignore during loss computation
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# Tokenize datasets
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)

# Debugging batch sizes
from torch.utils.data import DataLoader

batch_size = 4
train_loader = DataLoader(tokenized_train_dataset, batch_size=batch_size)

for batch in train_loader:
    # Convert 'input_ids' and 'labels' to PyTorch tensors
    # Use nested list comprehension to flatten 'input_ids' and 'labels'
    input_ids = torch.tensor([[item for sublist in batch['input_ids'] for item in sublist]])
    labels = torch.tensor([[item for sublist in batch['labels'] for item in sublist]])

    print(f"Input batch shape: {input_ids.shape}")
    print(f"Label batch shape: {labels.shape}")
    break



# Tokenize the train and validation datasets
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/655 [00:00<?, ? examples/s]

Map:   0%|          | 0/164 [00:00<?, ? examples/s]

Input batch shape: torch.Size([1, 1024])
Label batch shape: torch.Size([1, 324])


Map:   0%|          | 0/655 [00:00<?, ? examples/s]

Map:   0%|          | 0/164 [00:00<?, ? examples/s]

In [109]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


In [110]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./gpt2-lora-summarization",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True,  # Use mixed precision if supported
)


In [98]:
# outputs = model.generate(
#     input_ids,
#     max_new_tokens=50,  # Limits generated sequence length
#     pad_token_id=tokenizer.pad_token_id,
# )


In [99]:
from evaluate import load

rouge_metric = load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace out-of-range token IDs in labels
    labels = [
        [token_id if token_id < tokenizer.vocab_size else tokenizer.pad_token_id for token_id in label]
        for label in labels
    ]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute Rouge scores
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)

    # If the results are raw scores (float values), return them directly
    if isinstance(result[list(result.keys())[0]], float):
        return result
    else:
        # For structured results, extract 'fmeasure' or other metrics
        return {key: value['fmeasure'] for key, value in result.items()}


In [100]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [111]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


ValueError: Expected input batch_size (255) to match target batch_size (80).